# Imports

In [ ]:
import os
import sys 
import json
import pickle
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib

from scipy import stats
from pylab import rcParams
from scipy.stats import randint
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

rcParams['figure.figsize'] = (16, 9)
matplotlib.rcParams['figure.figsize'] = (16, 9)

In [ ]:
data = pd.read_csv('data/household_power_consumption.txt', sep=';', 
                   parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                   low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
data.head()

In [ ]:
data.describe()

# Exploratory data analysis

In [ ]:
for column in data.columns:
    plt.title(f'{column} (2 week duration)')
    data[column][:60*24*14].plot()
    plt.show()

Для простоты будем работать с **дневным** потреблением электричества (*Global active power*), посмотрим внимательнее на данные.

In [ ]:
data[['Global_active_power']].resample('D').apply(sum).plot(title='Total GAP per day', figsize=(12, 8)) 
plt.tight_layout()
plt.show()

## Задание: Что не так с этим графиком?

In [ ]:
data.isna().sum()

В данных пропусков примерно на 18 дней, интерполируем пропущенные значения.

In [ ]:
data.interpolate(method='time', inplace=True)
data.isna().sum()

In [ ]:
data['Global_active_power'].describe()

In [ ]:
data[['Global_active_power']].resample('D').apply(sum).plot(title='Total GAP per day', figsize=(12, 8)) 
plt.tight_layout()
plt.show()

# Простые бейзлайн решения для прогнозирования рядов

### Посчитаем ошибки прогноза:
MAPE (mean absolute percentage error) — это средняя абсолютная ошибка нашего прогноза. MAPE часто используется для оценки качества, поскольку эта величина относительная и по ней можно сравнивать качество даже на различных наборах данных.

$y_i$ - значение ряда в момент времени $i$.  
$\hat{y_i}$ - прогноз нашей модели в то же время.  

$e_i = y_i - \hat{y_i}$ - ошибка прогноза.  
$p_i = \frac{e_i}{y_i}$ - относительная ошибка прогноза.

$MAE = mean\space (\mid{e_i}\mid)$  
$MAPE = mean\space (\mid{p_i}\mid)$

### Задание: реализовать функции для подсчёта MAE, MAPE

In [ ]:
def mean_abs_error(y_true, y_pred):
    return <your code here>

In [ ]:
def mean_abs_percentage_error(y_true, y_pred):
     return <your code here>

## Прогнозирование вчерашним днём

In [ ]:
def plot_prediction(gt, pred, last_n=60, title='Graph'):
    plt.plot(pred[-last_n:], label='Prediction')
    plt.plot(gt[-last_n:], label='Ground Truth')
    plt.title(title)
    plt.grid(True)
    plt.legend()

In [ ]:
df = data['Global_active_power'].resample('D').apply(sum)
prediction = df.shift(1)

In [ ]:
plot_prediction(df, prediction)

In [ ]:
print(f'Naive MAE = {mean_abs_error(df[1:], prediction.dropna())}')
print(f'Naive MAPE = {mean_abs_percentage_error(df[1:], prediction.dropna())}')

## Скользящее среднее

In [ ]:
prediction = df.rolling(7).apply(np.mean).shift(1)

In [ ]:
plot_prediction(df, prediction)

In [ ]:
print(f'Moving average MAE = {mean_abs_error(df[7:], prediction.dropna())}')
print(f'Moving average MAPE = {mean_abs_percentage_error(df[7:], prediction.dropna())}')

## Экспоненциальные сглаживания

In [ ]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return pd.Series(result, index=series.index)

prediction = exponential_smoothing(df, 0.7)

In [ ]:
plot_prediction(df, prediction.shift(1))

In [ ]:
print(f'Exponential smoothing MAE = {mean_abs_error(df[1:], prediction.shift(1).dropna())}')
print(f'Exponential smoothing MAPE = {mean_abs_percentage_error(df[1:], prediction.shift(1).dropna())}')

## Взвешенное скользящее среднее

In [ ]:
def weighted_average(series, weights):
    if np.sum(weights) != 1:
        raise ValueError('Weights must sum to 1')
    weights.reverse()
    window_size = len(weights)
    w_fn = lambda x: np.sum(weights * x)
    return series.rolling(window_size).apply(w_fn)

prediction = weighted_average(df, [0.6, 0.2, 0.1, 0.07, 0.03])

In [ ]:
plot_prediction(df, prediction.shift(1))

In [ ]:
print(f'Weighted moving average MAE = {mean_abs_error(df[5:], prediction.shift(1).dropna())}')
print(f'Weighted moving average MAPE = {mean_abs_percentage_error(df[5:], prediction.shift(1).dropna())}')

# ARIMA

## Краткая теория по временным рядам

В классических задачах анализа данных предполагается что все наблюдение независимы, однако при прогнозировании временных рядов, мы наоборот, надеемся, что значения ряда в прошлом содержат информацию о его поведении в будущем.

### Компоненты временных рядов:

**Тренд** — плавное долгосрочное изменение уровня ряда.  
**Сезонность** — циклические изменения уровня ряда с постоянным
периодом.  
**Цикл** — изменения уровня ряда с переменным периодом (экономические
циклы, периоды солнечной активности).  
**Ошибка** — непрогнозируемая случайная компонента ряда.


### Стационарность:

Под [**стационарностью**](https://ru.wikipedia.org/wiki/Стационарность) понимают свойство процесса не менять своих статистических характеристик с течением времени, а именно постоянство матожидания, постоянство дисперсии (она же [гомоскедастичность](https://ru.wikipedia.org/wiki/Гомоскедастичность)) и независимость ковариационной функции от времени (должна зависеть только от расстояния между наблюдениями).

Для нас это означает, что временные ряды с трендом и сезонностью - нестационарны. Цикличность, тем не менее, оставляет ряд стационарным, потому что нельзя заранее предсказать, где будут находиться минимумы или максимумы ряда.  

Почему стационарность так важна? По стационарному ряду просто строить прогноз, так как мы полагаем, что его будущие статистические характеристики не будут отличаться от наблюдаемых текущих. Большинство моделей временных рядов так или иначе моделируют и предсказывают эти характеристики (например, матожидание или дисперсию), поэтому в случае нестационарности исходного ряда предсказания окажутся неверными. К сожалению, большинство временных рядов, с которыми приходится сталкиваться за пределыми учебных материалов, стационарными не являются, но с этим можно (и нужно) бороться.

Бороться с нестационарностью можно множеством способов - дифференцированием, выделением тренда и сезонности, сглаживаниями и различными преобразованиями (логарифмирование, Бокс-Кокс).

Формально проверить гипотезу о стационарности ряда можно с помощью теста [Дики-Фуллера](https://ru.wikipedia.org/wiki/Тест_Дики_—_Фуллера). Однако всегда нужно смотреть на ряд глазами, потому что тесты в ряде случаях могут ошибаться. 

## Задание: Какие из представленных ниже временных рядов стационарны?

<img src="./images/download.png"/>

Знаменитая картинка из [лекции](https://www.youtube.com/watch?v=u433nrxdf5k) Евгения Рябенко о временных рядах. 

## Модель

Немного о модели: [ARIMA](https://ru.wikipedia.org/wiki/ARIMA) (autoregressive integrated moving average). Существует [теорема Вольда](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%92%D0%BE%D0%BB%D0%B4%D0%B0), которая говорит о том, что любой стационарный ряд может быть описан моделью ARIMA, а это значит, что наша задача в первую очередь привести ряд к стационарному виду, после чего производить моделирование.

In [ ]:
import patsy
import statsmodels.api as sm

In [ ]:
df = data[['Global_active_power']].resample('D').apply(np.mean)
df.head()

In [ ]:
df.plot();

In [ ]:
decomposition = sm.tsa.seasonal_decompose(df['Global_active_power'])
fig = decomposition.plot()
plt.show()

print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['Global_active_power'])[1])

## Преобразование Бокса-Кокса

Это монотонное преобразование для данных, которое обычно используется для стабилизации дисперсии. Подробнее [здесь.](https://en.wikipedia.org/wiki/Power_transform) 

Для исходной последовательности $y = \{ y_1, \ldots, y_n \}, \quad y_i > 0, \quad i = 1,\ldots,n$ однопараметрическое преобразование Бокса-Кокса с параметром $\lambda$ определяется следующим образом:

$y_i^{\lambda} = \begin{cases}\frac{y_i^\lambda-1}{\lambda},&\text{if } \lambda \neq 0,\\ \log{(y_i)},& \text{if } \lambda = 0.\end{cases}$

Параметр $\lambda$ можно выбирать, максимизируя логарифм правдоподобия. Еще один способ поиска оптимального значения параметра основан на поиске максимальной величины коэффициента корреляции между квантилями функции нормального распределения и отсортированной преобразованной последовательностью.

In [ ]:
df['Global_active_power_log'] = np.log(df['Global_active_power'])
df['Global_active_power_log'].plot();
plt.ylabel(u'Logarithmic global active power')

print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['Global_active_power_log'])[1])

Выборочная автокорреляция - обычная корреляция между исходным рядом и его версией, сдвинутой на несколько отсчётов. Колличество отсчётов, на которое мы сдвигаем ряд называется **лагом автокорреляции**.  

$$\hat{\rho}_k = \frac{\sum_{t=k+1}^{T}(y_t - \bar{y})(y_{t-k}-\bar{y})}{\sum_{t=1}^{T}(y_t - \bar{y})^2} $$

### В каком диапазоне лежат значения автокорреляции? :)

Выборочная частичная автокорреляция:  

$$\hat{y}_t = * + * \cdot y_{t-1} + * \cdot y_{t_2} + \cdots + * \cdot y_{t-k+1} + \phi_{k} \cdot y_{t-k} + u_t$$

In [ ]:
ax = plt.subplot(2, 1, 1)
sm.graphics.tsa.plot_acf(df['Global_active_power_log'].dropna().values.squeeze(), lags=48, ax=ax)

ax = plt.subplot(2, 1, 2)
sm.graphics.tsa.plot_pacf(df['Global_active_power_log'].dropna().values.squeeze(), lags=48, ax=ax)
plt.show()

print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['Global_active_power_log'].dropna())[1])

In [ ]:
df['Global_active_power_log_week'] = df['Global_active_power_log'] - df['Global_active_power_log'].shift(7)
df['Global_active_power_log_diff_week'] = df['Global_active_power_log_week'] - df['Global_active_power_log_week'].shift(1)

ax = plt.subplot(2, 1, 1)
sm.graphics.tsa.plot_acf(df['Global_active_power_log_week'].dropna().values.squeeze(), lags=96, ax=ax)

ax = plt.subplot(2, 1, 2)
sm.graphics.tsa.plot_pacf(df['Global_active_power_log_week'].dropna().values.squeeze(), lags=96, ax=ax)
plt.show()

print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['Global_active_power_log_week'].dropna())[1])

In [ ]:
ax = plt.subplot(2, 1, 1)
sm.graphics.tsa.plot_acf(df['Global_active_power_log_diff_week'].dropna().values.squeeze(), lags=96, ax=ax)

ax = plt.subplot(2, 1, 2)
sm.graphics.tsa.plot_pacf(df['Global_active_power_log_diff_week'].dropna().values.squeeze(), lags=96, ax=ax)
plt.show()

print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['Global_active_power_log_diff_week'].dropna())[1])

In [ ]:
# Посмотрим в итоге на то, что у нас получилось.
df['Global_active_power_log_diff_week'].dropna().plot();

In [ ]:
ps, Ps = range(0, ), range(0, )
qs, Qs = range(0, ), range(0, )
d, D = 
lag = 

In [ ]:
day_ns = df['Global_active_power_log'].index.dayofyear
day_ns

In [ ]:
exog_season = patsy.dmatrix('bs(day_ns, df=8)')
print(exog_season.shape)
print(exog_season[:5])

exog_season2 = exog_season[:, 1:]  # bug/requirement: no constant in exog

In [ ]:
train_portion = 31
train_df, test_df = df[:-train_portion], df[-train_portion:]
train_exog, test_exog = exog_season2[:-train_portion], exog_season2[-train_portion:]

In [ ]:
import itertools
import warnings

parameters = itertools.product(ps, qs, Ps, Qs)
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')

for parameter in list(parameters):
    # Модель обучается не на всех наборах параметров
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['Global_active_power_log'], 
                                        order=(parameter[0], d, parameter[1]), 
                                        seasonal_order=(parameter[2], D, parameter[3], lag),
                                        exog=train_exog).fit(disp=-1)
    # Выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError as e:
        print('wrong parameters:', parameter)
        continue
    aic = model.aic
    # Сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_parameter = parameter
    results.append([parameter, model.aic])

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
result_table.sort_values(by='aic', ascending=True).head()

In [ ]:
train_df['model'] = np.exp(best_model.fittedvalues)
plot_prediction(train_df['Global_active_power'], train_df['model'], last_n=300)

In [ ]:
pred_values = np.exp(best_model.forecast(31, exog=test_exog))
plt.plot(np.concatenate([train_df['Global_active_power'].values[-200:], pred_values]));

In [ ]:
plot_prediction(test_df['Global_active_power'], pred_values)

In [ ]:
print(f'SARIMAX MAE = {mean_abs_error(pred_values, test_df["Global_active_power"].values)}')
print(f'SARIMAX MAPE = {mean_abs_percentage_error(pred_values, test_df["Global_active_power"].values)}')

In [ ]:
sarima_folder = os.path.join('models', 'sarima')
!mkdir -p {sarima_folder}

In [ ]:
best_model.save(os.path.join(sarima_folder, 'model.sa'))

In [ ]:
np.save(os.path.join(sarima_folder, 'exog'), test_exog)

# Fbprophet

Prophet был разработан для прогнозирования большого числа различных бизнес-показателей и строит достаточно хорошие default'ные прогнозы. Кроме того, он дает возможность, изменяя человеко-понятные параметры, улучшать прогноз и не требует от аналитиков глубоких знаний устройства предсказательных моделей.

Из минусов можно отметить достаточно небольшую точность предсказания "из коробки", для повышения качества придется повозиться с настройкой параметров.

[Статья](https://research.fb.com/prophet-forecasting-at-scale/) от Facebook про Prophet.  
[Ссылка](https://facebook.github.io/prophet/docs/quick_start.html) на официальную документацию.

In [ ]:
from fbprophet import Prophet

Библиотека **Prophet** имеет интерфейс похожий на *sklearn*, сначала мы создаем модель, затем вызываем у нее метод *fit* для тренировки и *predict* для предсказания. На вход методу *fit* библиотека принимает *dataframe* с двумя колонками:

**ds** — время, поле должно быть типа date или datetime,  
**y** — числовой показатель, который мы хотим предсказывать.

In [ ]:
df = data[['Global_active_power']].copy()
df = df.resample('D').apply(sum)
df.reset_index(inplace=True)
df.columns = ['ds', 'y']

In [ ]:
df.tail()

In [ ]:
prediction_size = 31

df = data[['Global_active_power']].copy()
df = df.resample('D').apply(sum)
df.reset_index(inplace=True)
df.columns = ['ds', 'y']
df.tail()

In [ ]:
train_df = df[:-prediction_size]

In [ ]:
model = Prophet(daily_seasonality=True)
model.fit(train_df);

Для того что бы построить предсказания методу *predict* нужно передать *dataframe* с количеством записей, равным периоду, на который нужно предсказать.

In [ ]:
future_dataframe = model.make_future_dataframe(periods=prediction_size, freq='D')
future_dataframe.tail()

In [ ]:
forecast = model.predict(future_dataframe)
forecast.tail()

In [ ]:
model.plot(forecast);

In [ ]:
model.plot_components(forecast);

In [ ]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [ ]:
cmp_df = make_comparison_dataframe(df, forecast)

mape = mean_abs_percentage_error(cmp_df['y'][-prediction_size:], cmp_df['yhat'][-prediction_size:])
mae = mean_abs_error(cmp_df['y'][-prediction_size:], cmp_df['yhat'][-prediction_size:])

print(f'Prophet MAPE = {mape},\nProphet MAE = {mae}')

In [ ]:
plot_prediction(cmp_df['y'], cmp_df['yhat'])

In [ ]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [ ]:
df_with_boxcox = df.copy().set_index('ds')
df_with_boxcox['y'], lambda_ = np.log(df_with_boxcox['y']), 0
df_with_boxcox.reset_index(inplace=True)

In [ ]:
pd.concat([df_with_boxcox, df.y], axis=1).head()

In [ ]:
train_df_wb = df_with_boxcox[:-prediction_size]

model_wb = Prophet(daily_seasonality=True)
model_wb.fit(train_df_wb);

In [ ]:
future_df_wb = model_wb.make_future_dataframe(periods=prediction_size, freq='D')
future_df_wb.tail()

In [ ]:
forecast_wb = model_wb.predict(future_df_wb)
forecast_wb.tail()

In [ ]:
model_wb.plot(forecast_wb);

In [ ]:
model_wb.plot_components(forecast_wb);

In [ ]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast_wb[column] = inverse_boxcox(forecast_wb[column], lambda_)

In [ ]:
cmp_df_wb = make_comparison_dataframe(df, forecast_wb)
mape_wb = mean_abs_percentage_error(cmp_df_wb['y'][-prediction_size:], cmp_df_wb['yhat'][-prediction_size:])
mae_wb = mean_abs_error(cmp_df_wb['y'][-prediction_size:], cmp_df_wb['yhat'][-prediction_size:])

print(f'MAPE = {mape:.2f}, MAPE with Box-Cox = {mape_wb:.2f}')
print(f'MAE = {mae:.2f}, MAE with Box-Cox = {mae_wb:.2f}')

In [ ]:
plot_prediction(cmp_df_wb['y'], cmp_df_wb['yhat'])

In [ ]:
!mkdir -p ./models/fbprophet/
import pickle
with open('models/fbprophet/model.pkl', 'wb') as f:
    pickle.dump(model, f)

# LSTM

**RNNS**
<img src='./images/RNN_types.jpeg'>

<img src="./images/RNN.png">

<img src='./images/RNNnotation.png'>

**LSTM - Long-Short-term Memory**

Данный вид рекуррентных нейронных сетей позволяет сохранять как длительные, так и коротковременные зависимости в последовательностях данных. Это происходит за счёт особой архитектуры сети, которую вы можете увидеть ниже  

<img src='./images/LSTM.png'>

В каждой ячейке имеется вход новых данных и вход предыдущего состояния. А также ячейка имеет свое "состояние", благодаря которому сеть и "запоминает" данные.  

$$f = \sigma(W_f [h_{t-1}; x_t] + b_f)$$  
$$i = \sigma(W_i [h_{t-1}; x_t] + b_i)$$  
$$o = \sigma(W_o [h_{t-1}; x_t] + b_o)$$  

$$\tilde c_{t} = tanh(W_h [h_{t-1}; x_t] + b_h)$$  
$$c_t = f \odot c_{t-1} + i \odot \tilde c_t$$  

$$h_t = o \odot tanh(c_t)$$

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
df_resample = data.resample('h').mean() 
df_resample.shape

In [ ]:
values = df_resample.values 

In [ ]:
values.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
reframed = series_to_supervised(scaled, 7, 1)

reframed.drop(reframed.columns[50:], axis=1, inplace=True)
print(reframed.head())

In [ ]:
values = reframed.values

n_train_time = 365*24*3
train = values[:n_train_time, :]
test = values[n_train_time:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape(<your code here>)
test_X = test_X.reshape(<your code here>)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 

In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(40, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(20, return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(10))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

In [ ]:
model = create_lstm_model()

In [ ]:
history = model.fit(train_X, train_y, epochs=20, batch_size=70,
                    validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
def plot_history(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    
# summarize history for loss
plot_history(history)

In [ ]:
def convert_predictions(predictions, test_X, test_y, scaler):
    test_X_ = test_X[:, 0, :]
    # invert scaling for forecast
    inv_yhat = np.concatenate((predictions, test_X_[:, -6:]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_y, test_X_[:, -6:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    return inv_y, inv_yhat
    
# make a prediction
yhat = model.predict(test_X)
inv_y, inv_yhat = convert_predictions(yhat, test_X, test_y, scaler)

In [ ]:
plot_prediction(inv_y, inv_yhat, last_n=200)

In [ ]:
print(f'LSTM MAE = {mean_abs_error(inv_y, inv_yhat)}')
print(f'LSTM MAPE = {mean_abs_percentage_error(inv_y, inv_yhat)}\n')

In [ ]:
def save_model(model, scaler, folder_name):
    model_folder = os.path.join('models', folder_name)
    model_json_path = os.path.join(model_folder, 'model.json')
    weights_path = os.path.join(model_folder, 'weights.hd5')
    scaler_path = os.path.join(model_folder, 'scaler.pkl')

    !mkdir -p {model_folder}

    with open(model_json_path, 'w') as f:
        json.dump(model.to_json(), f)
    model.save_weights(weights_path)

    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

In [ ]:
save_model(model, scaler, 'lstm_keras_h')

In [ ]:
def train_lstm(dataset, n_train_time):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(dataset)
    
    reframed = series_to_supervised(scaled, 7, 1)
    reframed.drop(reframed.columns[50:], axis=1, inplace=True)
    
    values = reframed.values

    train = values[:n_train_time, :]
    test = values[n_train_time:, :]

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 7, 7))
    test_X = test_X.reshape((test_X.shape[0], 7, 7))

    model = create_lstm_model()
    history = model.fit(train_X, train_y, epochs=20, batch_size=70,
                        validation_data=(test_X, test_y), verbose=2, shuffle=False)

    # summarize history for loss
    plot_history(history)
    
    yhat = model.predict(test_X)
    inv_y, inv_yhat = convert_predictions(yhat, test_X, test_y, scaler)    
    plot_prediction(inv_y, inv_yhat, last_n=200)
    
    print(f'LSTM MAE = {mean_abs_error(inv_y, inv_yhat)}')
    print(f'LSTM MAPE = {mean_abs_percentage_error(inv_y, inv_yhat)}\n')
    
    return model, scaler

In [ ]:
model, scaler = train_lstm(data.resample('d').mean(), 365*3)

In [ ]:
save_model(model, scaler, 'lstm_keras_d_mean')

In [ ]:
model, scaler = train_lstm(data.resample('d').sum(), 365*3)

In [ ]:
save_model(model, scaler, 'lstm_keras_d_sum')

In [ ]:
model, scaler = train_lstm(data.resample('h').mean(), 365*24*3)

# LightGBM

In [ ]:
import lightgbm as lgb

**LightGBM** это одна из самых популярных библиотек для градиентного бустинга. Основные её преимущества в том, что она эффективна по памяти и действительно быстрая (бонусом идёт возможность работать с большими массивами данных и sparse матрицами).

[Ссылка](https://lightgbm.readthedocs.io/en/latest/) на официальную документацию.

## Задание: придумайте признаки для модели

In [ ]:
def mean_encoding(data, cat_feature, real_feature):
    """
    Возвращает словарь, где ключами являются уникальные категории признака cat_feature, 
    а значениями - средние по real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

In [ ]:
def prepare_df(data, target_column, lag_start=1, lag_end=2, test_size=0.15):
    
    data = pd.DataFrame(data.copy())
    
    # считаем индекс в датафрейме, после которого начинается тестовый отрезок
    test_index = int(len(data) * (1 - test_size))
    
    # добавляем лаги исходного ряда в качестве признаков
    for column in data.columns:
        for i in range(lag_start, lag_end):
            data[f"{column}_t-{i}"] = data[column].shift(i)
        if column != target_column:
            data.drop(column, axis=1, inplace=True)
        
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data['is_weekend'] = data.weekday.isin([5, 6]) * 1
    
    # считаем средние только по тренировочной части, чтобы избежать лика
    data[f'{target_column}_weekday_average'] = list(map(mean_encoding(data[:test_index], 'weekday',
                                                                      target_column).get, data.weekday))
    data[f"{target_column}_hour_average"] = list(map(mean_encoding(data[:test_index], 'hour',
                                                                   target_column).get, data.hour))

    group = list(data1.groupby(['hour', f"{target_column}_hour_average"]).indices)
    h_mean = [m for h, m in group]
    group = list(data1.groupby(['weekday', f"{target_column}_weekday_average"]).indices)
    w_mean = [m for w, m in group]
    
    # выкидываем закодированные средними признаки
    data.drop(["hour", "weekday"], axis=1, inplace=True)

    
    data = data.dropna()
    data = data.reset_index(drop=True)
    
    
    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = data.loc[:test_index].drop([target_column], axis=1)
    y_train = data.loc[:test_index][target_column]
    X_test = data.loc[test_index:].drop([target_column], axis=1)
    y_test = data.loc[test_index:][target_column]
    
    return X_train, X_test, y_train, y_test, w_mean, h_mean

In [ ]:
X_train, X_test, y_train, y_test, w_mean, h_mean = prepare_df(data.resample('h').mean(),
                                                              "Global_active_power",
                                                              lag_end=7,
                                                              test_size=0.15)

In [ ]:
X_train.head()

In [ ]:
# Логарифмируем таргет (частный случай преобразования Бокса-Кокса)
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train_log)
lgb_test = lgb.Dataset(X_test, y_test_log, reference=lgb_train)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'learning_rate': 0.005,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                verbose_eval=False,
                valid_sets=lgb_test,
                num_boost_round=10000,
                early_stopping_rounds=50)

In [ ]:
# Не забываем про обратное преобразование
y_pred = np.exp(gbm.predict(X_test))

print(f'GBM MAE = {mean_abs_error(y_test, y_pred)}')
print(f'GBM MAPE = {mean_abs_percentage_error(y_test, y_pred)}\n')

In [ ]:
plot_prediction(y_test.values, y_pred, last_n=200)

In [ ]:
!mkdir -p ./models/light_gbm_h_mean/
gbm.save_model('models/light_gbm_h_mean/weights.gbm');
with open('models/light_gbm_h_mean/data.pkl', 'wb') as f:
    pickle.dump((w_mean, h_mean), f)

### Дублируем то же самое на дневных суммах и дневных средних

In [ ]:
def train_lgb(dataset):
    X_train, X_test, y_train, y_test, w_mean, h_mean = prepare_df(dataset,
                                                                  "Global_active_power",
                                                                  lag_end=7,
                                                                  test_size=0.15)
    y_train_log = np.log(y_train)
    y_test_log = np.log(y_test)

    lgb_train = lgb.Dataset(X_train, y_train_log)
    lgb_test = lgb.Dataset(X_test, y_test_log, reference=lgb_train)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'learning_rate': 0.005,
        'verbose': 0
    }

    gbm = lgb.train(params,
                    lgb_train,
                    verbose_eval=False,
                    valid_sets=lgb_test,
                    num_boost_round=10000,
                    early_stopping_rounds=50)
    # Не забываем про обратное преобразование
    y_pred = np.exp(gbm.predict(X_test))

    print(f'GBM MAE = {mean_abs_error(y_test, y_pred)}')
    print(f'GBM MAPE = {mean_abs_percentage_error(y_test, y_pred)}\n')
   
    
    return gbm, w_mean, h_mean

In [ ]:
gbm, w_mean, h_mean = train_lgb(data.resample('D').mean())
!mkdir -p ./models/light_gbm_d_mean/
gbm.save_model('models/light_gbm_d_mean/weights.gbm');
with open('models/light_gbm_d_mean/data.pkl', 'wb') as f:
    pickle.dump((w_mean, h_mean), f)

In [ ]:
gbm, w_mean, h_mean = train_lgb(data.resample('D').sum())
!mkdir -p ./models/light_gbm_d_sum/
gbm.save_model('models/light_gbm_d_sum/weights.gbm');
with open('models/light_gbm_d_sum/data.pkl', 'wb') as f:
    pickle.dump((w_mean, h_mean), f)